# READ THE CSV MAKE DF



In [4]:
import pandas as pd

df =pd.read_csv("/content/flipkart_com-ecommerce_sample.csv")
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
uniq_id,20000,20000,c2d766ca982eca8304150849735ffef9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crawl_timestamp,20000,371,2015-12-01 12:40:44 +0000,1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_url,20000,20000,http://www.flipkart.com/alisha-solid-women-s-c...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_name,20000,12676,TheLostPuppy Back Cover for Apple iPad Air,134,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_category_tree,20000,6466,"[""Jewellery >> Necklaces & Chains >> Necklaces""]",1567,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pid,20000,19998,JEAEGE8Q8GXYFTGU,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
retail_price,19922.0,NaN,NaN,NaN,2979.206104,9009.639341,35.0,666.0,1040.0,1999.0,571230.0
discounted_price,19922.0,NaN,NaN,NaN,1973.401767,7333.58604,35.0,350.0,550.0,999.0,571230.0
image,19997,18589,"[""http://img6a.flixcart.com/image/car-mat/m/t/...",45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_FK_Advantage_product,20000,2,False,19215,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Vector DB

## EMBEDD THE STRING AND IMAGE. MAKE A DICT TO MAINTAIN THE ID AND INDEX MATCHING

In [9]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Inputs: Text and Image
text1 = "Key Features of Alisha Solid Women's Cycling Shorts: Pack of 3, made of Cotton Lycra fabric, ideal for women. Designed as cycling shorts with a solid pattern. Gentle machine wash in lukewarm water; avoid bleach. Style Code: ALTHT_3P_21. Includes 3 shorts in the sales package."
text2= "Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3"
image_path = "/content/image.jpeg"  # Replace with your image path
image = Image.open(image_path)

image_path2 = "/content/image2.jpeg"  # Replace with your image path
image2 = Image.open(image_path2)
# Preprocess inputs
inputs = processor(text=[text1,text2], images=[image,image2], return_tensors="pt", padding=True)

# Generate embeddings
outputs = model(**inputs)
text_embeddings = outputs.text_embeds  # Text embedding
image_embeddings = outputs.image_embeds  # Image embedding

# Normalize embeddings (optional, for similarity search)
text_embeddings = text_embeddings / text_embeddings.norm(p=2, dim=-1, keepdim=True)
image_embeddings = image_embeddings / image_embeddings.norm(p=2, dim=-1, keepdim=True)


print("Text Embedding Shape:", text_embeddings.shape)
print("Image Embedding Shape:", image_embeddings.shape)


Text Embedding Shape: torch.Size([2, 512])
Image Embedding Shape: torch.Size([2, 512])


In [6]:
import requests
from io import BytesIO
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import math

count=0
SAMPLES=20

index_map={}
indx = 0
length=70
embedding =torch.empty(0)
for rows in df.iterrows():
  if count<SAMPLES:
    if count%10==0:
        print(count)
    count+=1
    images=[]
    for url in rows[1]['image'][1:-1].split(','):
        if url:
            try:
                # print(url)
                response = requests.get(url.replace('"',""), stream=True)
                response.raise_for_status()
                image = Image.open(BytesIO(response.content))
                image_path=url.split('/')[-1].split(".")[0]+".jpg"
                image.save(image_path)
                image = Image.open(image_path)
                images.append(image)
            except:
                continue
    if len(rows[1]['description'][0:70])>0 and len(images)>0:
        text = rows[1]['description']
        inputs = processor(text=[text[i:i + length] for i in range(0, len(text), length)], images=images, return_tensors="pt", padding=True)
        # Generate embeddings
        outputs = model(**inputs)
        text_embeddings = outputs.text_embeds  # Text embedding
        image_embeddings = outputs.image_embeds  # Image embedding
        index_map[rows[1]['uniq_id']]=[indx,indx+len(images)+math.ceil(len(text)/length)+1]
        indx += len(images)+math.ceil(len(text)/length)+1
        embedding = torch.cat((embedding,text_embeddings),0)
        embedding = torch.cat((embedding,image_embeddings),0)
print("EMBEDD SIZE:", embedding.shape)


0
10
EMBEDD SIZE: torch.Size([469, 512])


# QUERY

In [10]:
from sentence_transformers import SentenceTransformer
import torch
def search(query, embedding, index_map, dataframe,k=5 ):
    model = SentenceTransformer("clip-ViT-B-32")
    query_emb = model.encode(query)

    similarity_scores = []
    for key, value in index_map.items():
      score = model.similarity(embedding[value[0]:value[1]],query_emb).max()
      similarity_scores.append((key,score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    for uniq_id in similarity_scores[:k]:
        link =dataframe[dataframe['uniq_id']==uniq_id[0]]['image']
        print(f"Uniq ID: {uniq_id[0]}, Similarity Score: {uniq_id[1]} ")
        for l in link:
            print(l)
        print("#"*100)

In [11]:
search("leather black",embedding=embedding,index_map=index_map, dataframe =df)

NameError: name 'embedding' is not defined

In [12]:
from sentence_transformers import SentenceTransformer
import torch
def search_image(image_path, embedding, index_map, dataframe,k=5 ):
    image = Image.open(image_path)
    model = SentenceTransformer("clip-ViT-B-32")
    query_emb = model.encode(image)

    similarity_scores = []
    for key, value in index_map.items():
      score = model.similarity(embedding[value[0]:value[1]],query_emb).max()
      similarity_scores.append((key,score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    for uniq_id in similarity_scores[:k]:
        link =dataframe[dataframe['uniq_id']==uniq_id[0]]['image']
        print(f"Uniq ID: {uniq_id[0]}, Similarity Score: {uniq_id[1]} ")
        for l in link:
            print(l)
        print("#"*100)

In [34]:
search_image("/content/test.jpg",embedding=embedding,index_map=index_map, dataframe =df,k=2)

Uniq ID: c2d766ca982eca8304150849735ffef9, Similarity Score: 0.9487442970275879 
["http://img5a.flixcart.com/image/short/u/4/a/altht-3p-21-alisha-38-original-imaeh2d5vm5zbtgg.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5npdybzyt.jpeg", "http://img5a.flixcart.com/image/short/z/j/7/altght-7-alisha-38-original-imaeh2d5jsz2ghd6.jpeg"]
####################################################################################################
Uniq ID: ce5a6818f7707e2cb61fdcdbba61f5ad, Similarity Score: 0.9487442970275879 
["http://img6a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5cqtxe5gt.jpeg", "http://img5a.flixcart.com/image/short/z/j/7/altght-7-alisha-38-original-imaeh2d5jsz2ghd6.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/

In [35]:
search_image("/content/test2.jpg",embedding=embedding,index_map=index_map, dataframe =df,k=2)

Uniq ID: c29af37837afcaf44b779eca7c19295f, Similarity Score: 0.8790022730827332 
["http://img6a.flixcart.com/image/pet-shampoo/c/c/d/sh-df-07-sicons-200-1100x1100-imaeh3kvn25fznjt.jpeg", "http://img6a.flixcart.com/image/pet-shampoo/c/c/d/sh-df-07-sicons-200-original-imaeh3kvn25fznjt.jpeg"]
####################################################################################################
Uniq ID: c2d766ca982eca8304150849735ffef9, Similarity Score: 0.8700301647186279 
["http://img5a.flixcart.com/image/short/u/4/a/altht-3p-21-alisha-38-original-imaeh2d5vm5zbtgg.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5npdybzyt.jpeg", "http://img5a.flixcart.com/image/short/z/j/7/altght-7-alisha-38-original-imaeh2d5jsz2ghd6.jpeg"]
####################################################################################################


In [18]:
import math
def search_image_text(query,image_path, embedding, index_map, dataframe,k=5 ):
    query_search_results = search(query,embedding,index_map,dataframe,k=math.ceil(k/2))
    image_search_results = search_image(image_path,embedding,index_map,dataframe,k=math.ceil(k/2))



# Save pickle files

In [2]:
import pickle

def pickle_file(file_name,data):
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)
pickle_file('embeddings.pickle',embedding)
pickle_file('index_map.pickle',index_map)
pickle_file('dataframe.pickle',df)

In [ ]:
pickle_file('embeddings.pickle',embedding)
pickle_file('index_map.pickle',index_map)
pickle_file('dataframe.pickle',df)

In [3]:
def read_pickle_file(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

In [4]:
import pickle
embeddings_read = read_pickle_file('embeddings.pickle')
index_map_read = read_pickle_file('index_map.pickle')
df_read = read_pickle_file('dataframe.pickle')

In [5]:
embeddings_read.shape

torch.Size([469, 512])

In [6]:
len(index_map_read)

20

In [7]:
df_read.shape

(20000, 15)

In [13]:
search("leather black",embedding=embeddings_read,index_map=index_map_read, dataframe=df_read)

Uniq ID: 4044c0ac52c1ee4b28777417651faf42, Similarity Score: 0.8263331651687622 
["http://img5a.flixcart.com/image/short/5/z/c/altght4p-39-alisha-38-original-imaeh2d5ar6m55zd.jpeg", "http://img5a.flixcart.com/image/short/z/h/b/altght-9-alisha-36-original-imaeh2d5khxcdggw.jpeg", "http://img6a.flixcart.com/image/short/z/h/b/altght-9-alisha-36-original-imaeh2d5yj4cnjtz.jpeg", "http://img6a.flixcart.com/image/short/z/h/b/altght-9-alisha-36-original-imaeh2d55eacbgwg.jpeg", "http://img6a.flixcart.com/image/short/p/h/t/altht-3p-1-alisha-34-original-imaeh2d5jxvh4msy.jpeg"]
####################################################################################################
Uniq ID: 7f7036a6d550aaa89d34c77bd39a5e48, Similarity Score: 0.772011399269104 
["http://img6a.flixcart.com/image/sofa-bed/j/f/y/fhd112-double-foam-fabhomedecor-leatherette-black-leatherette-1100x1100-imaeh3gemjjcg9ta.jpeg", "http://img6a.flixcart.com/image/sofa-bed/j/f/y/fhd112-double-foam-fabhomedecor-leatherette-black-leat

In [14]:
search("fabric",embedding=embeddings_read,index_map=index_map_read, dataframe=df_read)

Uniq ID: 849ab05698081a374215d0b7d18047d0, Similarity Score: 0.853799045085907 
["http://img5a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-1100x1100-imaeh3getyhdnnxn.jpeg", "http://img5a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3getyhdnnxn.jpeg", "http://img5a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3gehkzxd64a.jpeg", "http://img6a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3getztbhygx.jpeg", "http://img6a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3ge5qgadxgh.jpeg", "http://img6a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3gefhxg2vxy.jpeg", "http://img5a.flixcart.com/image/sofa-bed/r/c/g/fhd132-double-foam-fabhomedecor-purple-purple-original-imaeh3gfmx3wjstu.jpeg"]
#############################

In [19]:
search_image_text(query="cycling shorts",image_path="/content/test2.jpg", embedding=embeddings_read, index_map=index_map_read, dataframe=df_read,k=5 )

Uniq ID: 0973b37acd0c664e3de26e97e5571454, Similarity Score: 0.8297384977340698 
["http://img5a.flixcart.com/image/short/6/2/h/altght-11-alisha-38-original-imaeh2d5uq9thnyg.jpeg", "http://img6a.flixcart.com/image/short/p/h/t/altht-3p-1-alisha-34-original-imaeh2d5jxvh4msy.jpeg", "http://img6a.flixcart.com/image/short/p/h/t/altht-3p-1-alisha-34-original-imaeh2d5yz9fcjxe.jpeg", "http://img6a.flixcart.com/image/short/p/h/t/altht-3p-1-alisha-34-original-imaeh2d5fa94gzuy.jpeg", "http://img6a.flixcart.com/image/short/x/5/f/altght-2-alisha-36-original-imaeh2d5xnueazgz.jpeg"]
####################################################################################################
Uniq ID: c2d766ca982eca8304150849735ffef9, Similarity Score: 0.8215460777282715 
["http://img5a.flixcart.com/image/short/u/4/a/altht-3p-21-alisha-38-original-imaeh2d5vm5zbtgg.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/j/